# Segmentation and Contours

In [15]:
import cv2
import numpy as np

# lets load a simple image with 3 black squares

image = cv2.imread('.\shape.jpg')
cv2.imshow('input image',image)
cv2.waitKey(0)

#Grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

#Find Canny edges
edged = cv2.Canny(gray,30,200)
cv2.imshow('canny edges',edged)
cv2.waitKey(0)

#Finding Contours
# use a copy of your image e.g edged.copy(),since findContours alters the image

_,contours,hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
cv2.imshow('canny edges after contouring',edged)
cv2.waitKey(0)
print("number of contours found="+ str(len(contours)))

# draw all contours 
# use '-1' as the 3rd parameter to draw all
cv2.drawContours(image,contours,-1,(0,255,0),3)

cv2.imshow('contours',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

number of contours found=4


In [19]:
# cv2.CHAIN_APPROX_NONE stores all the boundary points
# cv2.CHAIN_APPROX_SIMPLE provides just the start and end points


# Sorting Contours

In [1]:
import cv2
import numpy as np

# Load image
image=cv2.imread('.\shape.jpg')
cv2.imshow('0 - original image',image)
cv2.waitKey(0)

#Create a black image with ame dimensions as our loaded image
blank_image = np.zeros((image.shape[0],image.shape[1],3))

# Create a copy of our original image
original_image =image

#Grayscale our image
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Find canny edges
edged = cv2.Canny(gray,50,200)
cv2.imshow('1- canny edges',edged)
cv2.waitKey(0)

# find contours and print how many were found
_,contours,hierarchy = cv2.findContours(edged.copy(),
                                        cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
print("number of contours found="+ str(len(contours)))

#DRAW all contours
cv2.drawContours(blank_image,contours,-1,(0,255,0),3)
cv2.imshow('2-all contours over blank image',blank_image)
cv2.waitKey(0)

# draw all contours over blank image
cv2.drawContours(image,contours,-1,(0,255,0),3)
cv2.imshow('3-all contours ',image)


cv2.waitKey(0)
cv2.destroyAllWindows()

number of contours found=4


# sorting by area

In [21]:
# function we'll use to display contour area

def get_contour_areas(contours):
    #returns the areas of all contours as list
    all_areas=[]
    for cnt in contours:
        area = cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

#load the image
image = cv2.imread('.\shape.jpg')
original_image = image

# lets print the areas of the contours before sorting
print("contours areas before sorting", get_contour_areas(contours))

# sort contours large to small
sorted_contours = sorted(contours,key=cv2.contourArea,reverse=True)
# sorted_contours = sorted(contours,key=cv2.contoursArea,reverse=True) [:3]

print("contours areas after sorting",get_contour_areas(sorted_contours))

# Iterate over our  contours and draw one at a time
for c in sorted_contours:
    cv2.drawContours(original_image,[c],-1,(255,0,0),3)
    cv2.waitKey(0)
    cv2.imshow('Contours by area',original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

contours areas before sorting [17775.0, 19943.0, 33147.5, 9490.5]
contours areas after sorting [33147.5, 19943.0, 17775.0, 9490.5]


In [3]:
# functions we'll use for sorting by  position


def x_cord_contour(contours):
    #returns the x cordinate for the contour centroid
    if cv2.contourArea(contours) >10:
        M=cv2.moments(contours)
        return (int(M['m10']/M['m00']))
    
def label_Contour_center(image,c,i):
    # places a red circle on the centers of contours
    M=cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    
    # draw the contour number on the image
    cv2.circle(image,(cx,cy),10,(0,0,255),-1)
    return image

# load the image
iamge = cv2.imread('.\shape.jpg')
original_image = image.copy()

# compute center of mass or centroids and draw them on our image

for(i,c) in enumerate(contours):
    orig = label_Contour_center(image,c,i)
    
cv2.imshow("4 - contours center",image)
cv2.waitKey(0)

# sort by left to right using our X_cord_Contour function
contours_left_to_right = sorted(contours,key=x_cord_contour,reverse=False)

# labeling contours left to right

for (i,c) in enumerate(contours_left_to_right):
    cv2.drawContours(original_image,[c],-1,(0,0,255),3)
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(original_image,str(i+1),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('6 - left to right contour', original_image)
    cv2.waitKey(0)
    (x,y,w,h)=cv2.boundingRect(c)
    

# Approximating contours

In [6]:
import numpy as np
import cv2
image=cv2.imread('./house.png')
orig_image = image.copy()
cv2.imshow('original image',orig_image)
cv2.waitKey(0)

# Grayscale and binarize
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)


# find contours
_,contours,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_LIST,
                                      cv2.CHAIN_APPROX_NONE)

# Iterate through each contour and compute the bounding rectangle
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(orig_image,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.imshow('bounding rectangle',orig_image)
cv2.waitKey(0)

#iterate through each contour and compute the approx contour

for c in contours:
    #calculate accuarcy as a percent of the contour perimeter
    accuracy = 0.01 * cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,accuracy,True)
    cv2.drawContours(image,[approx],0,(0,255,0),2)
    cv2.imshow('approx ploy DP',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Matching contour shapes

In [46]:
import cv2
import numpy as np
# load the refernce image
template = cv2.imread('./shapes1.png',0)
#template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
cv2.imshow('template',template)
cv2.waitKey(0)

# load the imaghe which images we are trying to match
target =cv2.imread('./shape1_test.png')
target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)

# Threshold both images first before using cv2.findContours
ret,thresh1 = cv2.threshold(template,127,255,0)
ret,thresh2 = cv2.threshold(target_gray,127,255,0)

# find contours in template
_,contours,hierarchy = cv2.findContours(thresh1,cv2.RETR_CCOMP,
                                        cv2.CHAIN_APPROX_SIMPLE)

# we need to sort the contours by area so we can remove thelargest contour which is
# is the image outline

sorted_contours = sorted(contours,key=cv2.contourArea,reverse=True)

# we extract the second largest contour which will be our template contour 
template_contour = contours[1]

#extract contours from second target image
_,contours,hierarchy = cv2.findContours(thresh2,cv2.RETR_CCOMP,
                                        cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
    #Iterate through each contour in the target image and use cv2.matchshapes to 
    #compare contour shapes
    match =cv2.matchShapes(template_contour,c,2,0.0)
    print (match)
    # if the match value is less than 0.15 we 
    if match < 0.15:
        closest_contour = c
        break
    else:
        closest_contour = []
        
cv2.drawContours(target,[closest_contour],-1,(0,255,0),3)
cv2.imshow('output',target)
cv2.waitKey()
cv2.destroyAllWindows()

0.2452531901514008
1.3476111233699948
0.09783837174596288


# Line detection

In [126]:
import cv2
import numpy as np
import imutils

image = cv2.imread('./sudoku.png')
resized = imutils.resize(image,width=600)

# grayscale and canny edges extracted 
gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,100,170,apertureSize = 3)

#run houghlines using a rho accuracy of 1 pixel
#theta accuracy of np.pi / 180 which is 1 degree
#our line threshold is set to 240(number of points on line)

lines = cv2.HoughLines(edges, 1, np.pi / 180,200)

# we iterate through each line and convert it to the format required by cv.lines 
# (i.e requiring end points)
for line in lines:
    rho,theta =line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(resized, (x1,y1),(x2,y2),(0,255,0),2)

#cv2.imwrite('houghlines3.jpg',img)
        
    
cv2.imshow('Hough lines', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()



# Probabilistic Hough lines

In [136]:
import cv2
import numpy as np


image = cv2.imread('./sudoku.jpg')
resized = imutils.resize(image,width=500)
image=resized
# grayscale and canny edges extracted 
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,100,170,apertureSize = 3)

# Again we use the same rho and theta accuracies 
# however we specific a minimum vote (pts along line) of 100
# and min line length of 5 pixels and max gap between lines of 10 pixels

lines = cv2.HoughLinesP(edges,1,np.pi / 180 , 150 ,5,10)
#print(lines.shape)

for line in lines:
    x1,y1,x2,y2 = line[0]
    cv2.line(image,(x1,y1),(x2,y2),(0,255,0),3)
    
cv2.imshow('Probabilistic Hough lines', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# circle detection

In [137]:
import cv2
import numpy as np


image = cv2.imread('./bottlecap.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blur = cv2.medianBlur(gray, 5)
circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1.5, 10)

for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(image,(i[0], i[1]), i[2], (255, 0, 0), 2)
      
    # draw the center of the circle
    cv2.circle(image, (i[0], i[1]), 2, (0, 255, 0), 5)

cv2.imshow('detected circles', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Blob detection

In [141]:
image = cv2.imread('./sunflower.png',cv2.IMREAD_GRAYSCALE)


# set up the detector with default parameters.
detector = cv2.SimpleBlobDetector_create()

#detect blobs
keypoints = detector.detect(image)

# draw detected blobs as red circles.
#cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle
#corresponds to the size of blob

blank = np.zeros((1,1))
blobs = cv2.drawKeypoints(image,keypoints,blank,(0,255,255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('blobs',blobs)
cv2.waitKey(0)
cv2.destroyAllWindows()